In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mask-dataset/Dataset/test/test/without_mask/208.jpg
/kaggle/input/mask-dataset/Dataset/test/test/without_mask/89.jpg
/kaggle/input/mask-dataset/Dataset/test/test/without_mask/58.jpg
/kaggle/input/mask-dataset/Dataset/test/test/without_mask/436.jpg
/kaggle/input/mask-dataset/Dataset/test/test/without_mask/270.jpg
/kaggle/input/mask-dataset/Dataset/test/test/without_mask/143.jpg
/kaggle/input/mask-dataset/Dataset/test/test/without_mask/446.jpg
/kaggle/input/mask-dataset/Dataset/test/test/without_mask/440.jpg
/kaggle/input/mask-dataset/Dataset/test/test/without_mask/237.jpg
/kaggle/input/mask-dataset/Dataset/test/test/without_mask/295.jpg
/kaggle/input/mask-dataset/Dataset/test/test/without_mask/30.jpg
/kaggle/input/mask-dataset/Dataset/test/test/without_mask/334.jpg
/kaggle/input/mask-dataset/Dataset/test/test/without_mask/10.jpg
/kaggle/input/mask-dataset/Dataset/test/test/without_mask/54.jpg
/kaggle/input/mask-dataset/Dataset/test/test/without_mask/0.jpg
/kaggle/input/mas

In [4]:
# import the neccessary libraries
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.models import Sequential, Model,load_model
from tensorflow.keras.layers import Conv2D,Input,ZeroPadding2D, BatchNormalization,Activation,MaxPooling2D,Flatten, Dense,Dropout
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
# import imutils
import numpy as np

In [5]:
#Build a neural network
model = Sequential()
model.add(Conv2D(32,(3,3),activation ='relu',input_shape =(150,150,3)))
model.add(Conv2D(32,(3,3),activation ='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3),activation ='relu'))
model.add(Conv2D(64,(3,3),activation ='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(100,activation='relu'))
model.add(Dense(2,activation ='softmax'))
model.compile(optimizer ='adam',loss = 'binary_crossentropy', metrics =['acc'])

2021-09-14 05:48:48.569193: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-14 05:48:48.572828: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-14 05:48:48.621883: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-14 05:48:48.622535: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-09-14 05:48:48.622591: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-09-14 05:48:48.665514: I tensorflow/stream_executor/platform/def

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 146, 146, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 69, 69, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 34, 34, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 73984)             0

In [7]:
Train_dir ="../input/mask-dataset/Dataset/train/train"
train_data = ImageDataGenerator(rescale=1.0/255,
                              rotation_range=40,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              shear_range=0.2,
                              zoom_range=0.2,
                              horizontal_flip=True,
                              fill_mode='nearest')
train_gen = train_data.flow_from_directory(Train_dir,
                                          batch_size=25,
                                          target_size = (150,150))
validation_dir ="../input/mask-dataset/Dataset/test/test"
valid_data = ImageDataGenerator(rescale=1.0/255)
valid_gen = valid_data.flow_from_directory(validation_dir,
                                          batch_size=25,
                                          target_size=(150,150))

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [8]:
# initialize a callback checkpoint to keep saving the best model after each epoch while training
checkpoint = ModelCheckpoint('model2-{epoch:03d}.h5',monitor='val_loss',verbose=0,save_best_only=True,mode ='auto')

In [9]:
#train the model
history = model.fit_generator(train_gen,epochs=50,validation_data=valid_gen,callbacks=[checkpoint])

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021-09-14 05:49:08.471524: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-14 05:49:08.475255: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000140000 Hz


Epoch 1/50


2021-09-14 05:49:09.047444: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-09-14 05:49:09.896967: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-09-14 05:49:09.926981: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


53/53 [==============================] - 25s 354ms/step - loss: 0.7566 - acc: 0.5465 - val_loss: 0.2499 - val_acc: 0.9381
Epoch 2/50
53/53 [==============================] - 14s 262ms/step - loss: 0.3891 - acc: 0.8466 - val_loss: 0.1181 - val_acc: 0.9794
Epoch 3/50
53/53 [==============================] - 13s 255ms/step - loss: 0.3350 - acc: 0.8643 - val_loss: 0.0673 - val_acc: 0.9845
Epoch 4/50
53/53 [==============================] - 14s 272ms/step - loss: 0.2350 - acc: 0.9103 - val_loss: 0.0559 - val_acc: 0.9845
Epoch 5/50
53/53 [==============================] - 14s 258ms/step - loss: 0.1764 - acc: 0.9408 - val_loss: 0.1080 - val_acc: 0.9433
Epoch 6/50
53/53 [==============================] - 14s 266ms/step - loss: 0.1616 - acc: 0.9482 - val_loss: 0.1071 - val_acc: 0.9485
Epoch 7/50
53/53 [==============================] - 14s 262ms/step - loss: 0.2415 - acc: 0.9153 - val_loss: 0.1170 - val_acc: 0.9485
Epoch 8/50
53/53 [==============================] - 14s 260ms/step - loss: 0.206

In [10]:
#Build a neural network
model1 = Sequential()
model1.add(Conv2D(64,(3,3),activation ='relu',input_shape =(150,150,3)))
model1.add(Conv2D(64,(3,3),activation ='relu'))
model1.add(MaxPooling2D(2,2))
model1.add(Conv2D(32,(3,3),activation ='relu'))
model1.add(Conv2D(32,(3,3),activation ='relu'))
model1.add(MaxPooling2D(2,2))
model1.add(Flatten())
model1.add(Dropout(0.5))
model1.add(Dense(50,activation='relu'))
model1.add(Dense(2,activation ='softmax'))
model1.compile(optimizer ='adam',loss = 'binary_crossentropy', metrics =['acc'])

In [11]:
#train the model
history = model1.fit_generator(train_gen,epochs=50,validation_data=valid_gen)

Epoch 1/50
53/53 [==============================] - 15s 261ms/step - loss: 0.6201 - acc: 0.6201 - val_loss: 0.1058 - val_acc: 0.9639
Epoch 2/50
53/53 [==============================] - 14s 267ms/step - loss: 0.2870 - acc: 0.8844 - val_loss: 0.0827 - val_acc: 0.9536
Epoch 3/50
53/53 [==============================] - 13s 254ms/step - loss: 0.2213 - acc: 0.9170 - val_loss: 0.0411 - val_acc: 0.9897
Epoch 4/50
53/53 [==============================] - 14s 262ms/step - loss: 0.2809 - acc: 0.8969 - val_loss: 0.0573 - val_acc: 0.9845
Epoch 5/50
53/53 [==============================] - 14s 256ms/step - loss: 0.1862 - acc: 0.9374 - val_loss: 0.1596 - val_acc: 0.9175
Epoch 6/50
53/53 [==============================] - 14s 259ms/step - loss: 0.1675 - acc: 0.9398 - val_loss: 0.0287 - val_acc: 1.0000
Epoch 7/50
53/53 [==============================] - 14s 264ms/step - loss: 0.1821 - acc: 0.9322 - val_loss: 0.0828 - val_acc: 0.9845
Epoch 8/50
53/53 [==============================] - 14s 256ms/step - 